In [ ]:
# nodes in a netlist are components that need to be connected to other specific nodes 
# SIMILARLY - vertiports in a region need to be placed so that it can connect to another region, this stipulates on certain assumptions as follows -
# 1) vertiport needs to be placed in a region that has demand,
# 2) just like bus stops vertiports should be selected to meet demands and form loops  
#  - this means we need to assume that - we have to place a certain vertiport in a certain region (requirement of design)

# NOTE - on chip placement from netlist - chips from netlist are placed sequentially (why? is it possible to place the chips in parallel? why not parallel?)



In [ ]:
# MDP formulation 

# state space
# chip placement                                        VS               vertiport 
# every possible partial placement of chip from netlist          (possible equivalent) every possible placement of a vertiport in a zone (assumption - map is discritized, and certain zones in map must have vertiport) 



# action space
# chip placement                                                                        VS                              vertiport
# set of all locations where current chip can be placed on the canvas without certain violations                set/region of location where a vertiport needs to be placed without violating RA 


# transition function 

# reward function 
# chip placement                                                                                    VS                              vertiport
# zero for all action except last, negative weighted sum of proxy wire length, and congestion                           (similar idea) negative of distance between vertiports, and congestion
#                                                                                                                                      1) How to measure congestion
#                                                                                                                                      2) I think the negative distance is to balance the congestion
#                                                                                                                                      3) ?? OTHER METRIC FOR REWARD ?? 



In [ ]:
# the vertiport design problem will start -
# 1) empty map - and a list of vertiports that need to be placed in specific regions (assumption - WE know the region, but we do not know the exact location ) 
# 2) for vertiport design problem - FIRST, the vertiport_RL will place the vertiports, 
#                                   SECOND, after vertiports are placed, the REWARD of vertiport_RL will be calculated by running the simulator, after placing the last vertiport

# POINTS to think about 
# 1) how many rows and columns for discritizing the map
# 2) selection of vertiports (how do we decide which vertiports to start with)
# 3) density - two vertiports cannot be within certain grid units of one another - for this a mask will be place on the grid, masking RA, and placed vertiport and region around vertiport
#   

In [ ]:
# sound will be included in the vertiport design problem. 


## Vertiport design problem 


Approaching the problem from aspects we know are true with high certainty 


1. A, action space is either add a vertiport at a location, or delete a vertiport from a location
    - so action is clearly defined, either add or delete vertiport at location 
2. P(s'|s,a), the state transition is clear, its deterministic state transition, once all the vertiports are placed, then we will run simulation to collect statistics
    - the transition from one state, a state is the current location of all vertiports and deterministic transition is vertiports in new location
3. R(s,a,s'), the reward function 
    - EDGE attr: time_factor: time taken/nominal time, nominal time is the theoretical time it would take a UAV to reach its destination this theoretical time is distance/avg_speed
    - EDGE attr: distance_factor: distance covered/ point to point distance
    - EDGE attr: NMAC count
    - EDGE attr: RA violation 

    INSPIRATION FROM QUEUEING THEORY (arrival rate, service rate,....) <- these would be the node attributes i can assume both these are following possion distributions, and the parameters of arrival are learned from data, 
    and the departure rate is assumed based on some heuristic. Then I can use these attributes as well in the reward function. 

    node features will be determined from queueing theory use almost all attributes from queing theory


4. Update - 
    - GCN will help us update Node using edges
    - GCN will help us update the edges using nodes
    - applying either MAX or AVG pooling will provide us with complete overview of the network, and that vector can then be passed to the policy/value function of RL algo 


Thoughts on MCTS (Monte Carlo Tree Search)

Needs discrete state and action space, if not discrete then needs to made discrete. 
Using this method will not be possible due to huge state space/ action space, since MCTS will try to sample untried actions before retrying actions that have been tried before. 
This will lead to really slow updates combined with running simulation to obtain statistics. Now MCTS requires some form of value for each node, this would mean 
I would have to define a function that would take these natural attributes and form a non-linear function, which may introduce bias.




Three possible ways to solve the vertiport design problem - 
1) MCTS - 

First iteration of MCTS, start at vertiport, choose one waypoint or multiple waypoints, and from that waypoint(s) connect to end vertiport. 
Second iteration of MCTS, 
assumptions of this method,
    1. vertiport locations on discrete address (commercial building, residential area, etc) - discrete locations will be selected from locations provided by OSMNX 
    2. A rectangular grid will be overlayed on top of austin map
        - each grid point on the austin map is a possible waypoint 
        - the MCTS algo will choose a set of waypoints and build a route which will be handed to UAVs for motion 
        - for each route the following **metrics** will be calculated 
            - time_factor : sum over each subsection 
            - distance_factor : sum over each subsection 
            - NMAC count
            - RA violation
            - arrival rate
            - service rate
            - $\sigma$
    3. need to come up with heuristic for route selection 
        some possible heuristics:
        - explore waypoints that are facing the end vertiports first, 
        - 

MCTS process - for MCTS a states are 
- vertiport pairs and the waypoint between them, and associated metrics described above
- additional information that will be appended to each node (NODE in MCTS)  
2) GNN - RL 
assumptions for this method, 
    1. edges of graph:
        - EDGE attr: time_factor: time taken/nominal time, nominal time is the theoretical time it would take a UAV to reach its destination this theoretical time is distance/avg_speed
        - EDGE attr: distance_factor: distance covered/ point to point distance
        - EDGE attr: NMAC count
        - EDGE attr: RA violation
    2. nodes of the graph: 
        - arrival rate
        - service rate

with the above we can form a graph for GNN methods where a set of vertiports will be selected at random to start motion simulation. End of motion simulation we will calculate the values of nodes and edges. 
use these values and perform message passing to update the nodes and edges of graph. then perform pooling to aggregate the information in the graph into a vector. Then pass the vector to RL to choose new vertiports. 
Process will continue *up to 'convergence'*. 


